### Spark ML Quiz
* 문제: 맨 아래 셀에 코드를 작성하여 predict_test dataframe에서 Outcome과 prediction이 일치하는 샘플의 비율, 즉 정확도를 계산하시오

In [1]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

### Data loading

일단 그냥 읽어봄

In [2]:
raw_data = spark.read.format("csv").option("header","true").load("diabetes.csv")
raw_data.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [3]:
raw_data.printSchema()

root
 |-- Pregnancies: string (nullable = true)
 |-- Glucose: string (nullable = true)
 |-- BloodPressure: string (nullable = true)
 |-- SkinThickness: string (nullable = true)
 |-- Insulin: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- DiabetesPedigreeFunction: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Outcome: string (nullable = true)



In [4]:
raw_data.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



### Data loading

schema를 지정하여 읽음

In [5]:
from pyspark.sql.types import *

schema = StructType([
    StructField('Pregnancies',FloatType(),True),
    StructField('Glucose',FloatType(),True),
    StructField('BloodPressure',FloatType(),True),
    StructField('SkinThickness',FloatType(),True),
    StructField('Insulin',FloatType(),True),
    StructField('BMI',FloatType(),True),
    StructField('DiabetesPedigreeFunction',FloatType(),True),
    StructField('Age',IntegerType(),True),
    StructField('Outcome',IntegerType(),True)
])

raw_data=spark.read.format("csv").option("header","true").schema(schema).load("diabetes.csv")

In [6]:
raw_data.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



### describe(*cols)
Computes basic statistics for numeric and string columns.

This include count, mean, stddev, min, and max. If no columns are given, this function computes statistics for all numerical or string columns.

Note This function is meant for exploratory data analysis, as we make no guarantee about the backward compatibility of the schema of the resulting DataFrame.

In [7]:
raw_data.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------------+------------------+------------------+
|summary|       Pregnancies|          Glucose|     BloodPressure|     SkinThickness|           Insulin|              BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------------+------------------+------------------+
|  count|               768|              768|               768|               768|               768|              768|                     768|               768|               768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.99257813890775|      0.4718763029280429|33.240885416666664|0.3489583333333333|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.9522175

In [8]:
raw_data.describe('Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI').show()

+-------+-----------------+------------------+------------------+------------------+-----------------+
|summary|          Glucose|     BloodPressure|     SkinThickness|           Insulin|              BMI|
+-------+-----------------+------------------+------------------+------------------+-----------------+
|  count|              768|               768|               768|               768|              768|
|   mean|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.99257813890775|
| stddev|31.97261819513622|19.355807170644777|15.952217567727642|115.24400235133803|7.884160293010772|
|    min|              0.0|               0.0|               0.0|               0.0|              0.0|
|    max|            199.0|             122.0|              99.0|             846.0|             67.1|
+-------+-----------------+------------------+------------------+------------------+-----------------+



### summary(*statistics)
Computes specified statistics for numeric and string columns. Available statistics are: - count - mean - stddev - min - max - arbitrary approximate percentiles specified as a percentage (eg, 75%)

If no statistics are given, this function computes count, mean, stddev, min, approximate quartiles (percentiles at 25%, 50%, and 75%), and max.

Note This function is meant for exploratory data analysis, as we make no guarantee about the backward compatibility of the schema of the resulting DataFrame.

In [9]:
raw_data.select('Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI').summary().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+
|summary|          Glucose|     BloodPressure|     SkinThickness|           Insulin|              BMI|
+-------+-----------------+------------------+------------------+------------------+-----------------+
|  count|              768|               768|               768|               768|              768|
|   mean|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.99257813890775|
| stddev|31.97261819513622|19.355807170644777|15.952217567727642|115.24400235133803|7.884160293010772|
|    min|              0.0|               0.0|               0.0|               0.0|              0.0|
|    25%|             99.0|              62.0|               0.0|               0.0|             27.3|
|    50%|            117.0|              72.0|              23.0|              29.0|             32.0|
|    75%|            140.0|              80.0|              32.0|        

### 결측 값을 하나라도 갖는 row의 개수 카운트 하기

결측값이 None으로 표현되어 있는지 확인해 보기

In [10]:
raw_data.rdd.map(
    lambda row: 1 if sum([c == None for c in row]) > 0 else 0
).reduce(
    lambda x, y: x+y
)

0

### 결측 값을 하나라도 갖는 row의 개수 카운트 하기

0으로 표현된 결측값을 하나라도 갖는 row의 개수 세어보기

In [11]:
prep_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

raw_data.rdd.map(
    lambda row: 1 if sum([row[c] == 0 for c in prep_cols]) > 0 else 0
).reduce(
    lambda x, y: x+y
)

376

### Null zero값을 None으로 치환하기

In [12]:
import pyspark.sql.functions as fn

prep_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for c in prep_cols:
    raw_data = raw_data.withColumn(c, fn.when(fn.col(c) == 0, None).otherwise(fn.col(c)))

In [13]:
raw_data.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|        6.0|  148.0|         72.0|         35.0|   null|33.6|                   0.627| 50|      1|
|        1.0|   85.0|         66.0|         29.0|   null|26.6|                   0.351| 31|      0|
|        8.0|  183.0|         64.0|         null|   null|23.3|                   0.672| 32|      1|
|        1.0|   89.0|         66.0|         23.0|   94.0|28.1|                   0.167| 21|      0|
|        0.0|  137.0|         40.0|         35.0|  168.0|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



### 컬럼 별로 결측값의 비율 확인해보기

In [14]:
import pyspark.sql.functions as fn

prep_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

raw_data.select(*[
    (fn.count(c) / fn.count('*')).alias(c + '_missing') for c in prep_cols
]).show()

+------------------+---------------------+---------------------+------------------+------------------+
|   Glucose_missing|BloodPressure_missing|SkinThickness_missing|   Insulin_missing|       BMI_missing|
+------------------+---------------------+---------------------+------------------+------------------+
|0.9934895833333334|   0.9544270833333334|   0.7044270833333334|0.5130208333333334|0.9856770833333334|
+------------------+---------------------+---------------------+------------------+------------------+



### Imputing Missing Values

In [15]:
from pyspark.ml.feature import Imputer

prep_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

imputer=Imputer(inputCols=prep_cols,outputCols=prep_cols)
model=imputer.fit(raw_data)
raw_data=model.transform(raw_data)
raw_data.show(5)

+-----------+-------+-------------+-------------+---------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|  Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+---------+----+------------------------+---+-------+
|        6.0|  148.0|         72.0|         35.0|155.54822|33.6|                   0.627| 50|      1|
|        1.0|   85.0|         66.0|         29.0|155.54822|26.6|                   0.351| 31|      0|
|        8.0|  183.0|         64.0|     29.15342|155.54822|23.3|                   0.672| 32|      1|
|        1.0|   89.0|         66.0|         23.0|     94.0|28.1|                   0.167| 21|      0|
|        0.0|  137.0|         40.0|         35.0|    168.0|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+---------+----+------------------------+---+-------+
only showing top 5 rows



### Logistic Regression 분석을 위한 준비 1. Feature vector생성

In [16]:
# 예측해야할 class 속성 값 Outcome을 제외하고 나머지가 feature
cols=raw_data.columns
cols.remove("Outcome")

In [17]:
# VectorAssembler를 이용해 dataframe의 row를 vector로 변환
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=cols,outputCol="features")
raw_data=assembler.transform(raw_data)

In [18]:
raw_data.select("features").show(5, truncate=False)

+-----------------------------------------------------------------------------------------------+
|features                                                                                       |
+-----------------------------------------------------------------------------------------------+
|[6.0,148.0,72.0,35.0,155.5482177734375,33.599998474121094,0.6269999742507935,50.0]             |
|[1.0,85.0,66.0,29.0,155.5482177734375,26.600000381469727,0.35100001096725464,31.0]             |
|[8.0,183.0,64.0,29.153419494628906,155.5482177734375,23.299999237060547,0.671999990940094,32.0]|
|[1.0,89.0,66.0,23.0,94.0,28.100000381469727,0.16699999570846558,21.0]                          |
|[0.0,137.0,40.0,35.0,168.0,43.099998474121094,2.2880001068115234,33.0]                         |
+-----------------------------------------------------------------------------------------------+
only showing top 5 rows



### Logistic Regression 분석을 위한 준비 2. Normalization

In [19]:
from pyspark.ml.feature import StandardScaler

standardscaler=StandardScaler(
    inputCol="features", outputCol="scaled_features", withStd=True, withMean=False)
raw_data=standardscaler.fit(raw_data).transform(raw_data)
raw_data.select("features","scaled_features").show(5)

+--------------------+--------------------+
|            features|     scaled_features|
+--------------------+--------------------+
|[6.0,148.0,72.0,3...|[1.78063837321943...|
|[1.0,85.0,66.0,29...|[0.29677306220323...|
|[8.0,183.0,64.0,2...|[2.37418449762590...|
|[1.0,89.0,66.0,23...|[0.29677306220323...|
|[0.0,137.0,40.0,3...|[0.0,4.5012560836...|
+--------------------+--------------------+
only showing top 5 rows



### Logistic Regression 분석을 위한 준비 3. Train data set과 Test data set 생성

In [20]:
train, test = raw_data.randomSplit([0.8, 0.2], seed=37)

In [21]:
train.count()

608

In [22]:
test.count()

160

### Logistic Regression 돌리기

In [23]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    labelCol="Outcome",
    featuresCol="scaled_features",
    maxIter=10
)
model=lr.fit(train)
predict_train=model.transform(train)
predict_test=model.transform(test)
predict_test.select("Outcome","prediction").show(10)

+-------+----------+
|Outcome|prediction|
+-------+----------+
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      1|       0.0|
|      0|       0.0|
|      0|       0.0|
+-------+----------+
only showing top 10 rows



In [24]:
predict_test.select("Outcome","rawPrediction", "probability", "prediction").show(1, truncate=False)

+-------+----------------------------------------+--------------------------------------+----------+
|Outcome|rawPrediction                           |probability                           |prediction|
+-------+----------------------------------------+--------------------------------------+----------+
|0      |[2.9514000261074718,-2.9514000261074718]|[0.95032961588646,0.04967038411353985]|0.0       |
+-------+----------------------------------------+--------------------------------------+----------+
only showing top 1 row



### 아래 셀에 predict_test의 예측결과의 정확도를 계산하는 코드를 넣으시오

In [25]:
hit = predict_test.rdd.map(lambda row: 1 if row['Outcome'] == row['prediction'] else 0).reduce(lambda x, y: x+y)
testsize = predict_test.count()
print("precision = {}".format(hit / testsize))

precision = 0.78125
